# 1. Setup

# 2. Data Preprocessing
This is some explanation of feature present in the dataset

## 2.1 Numerical features: Total of 18 features
+ EHQ_EHQ_Total: laterality index score (float) || -100 = 10th left, −28 ≤ LI < 48 = middle, 100 = 10th right"
+ ColorVision_CV_Score: color vision test score (int)
+ MRI_Track_Age_at_Scan: Age at time of MRI scan (float)

### ALABAMA PARENTING QUESTIONAIRE - PARENT REPORT (INT)
+ APQ_P_APQ_P_CP: Reflects the frequency or severity of corporal punishment used by parents
+ APQ_P_APQ_P_ID: Measures inconsistency in parental discipline
+ APQ_P_APQ_P_INV: Indicates the level of parental involvement in the child’s life
+ APQ_P_APQ_P_OPD: Other Discipline Practices Score (Not factored into total score but provides item level information)
+ APQ_P_APQ_P_PM: Reflects how well a parent monitors and supervises their child
+ APQ_P_APQ_P_PP: Captures the extent of positive reinforcement and supportive parenting

### Strength and Difficulties Questionnaire (INT)
+ SDQ_SDQ_Conduct_Problems: Measures behavioral issues related to rule-breaking or aggression (higher score = more prone to ADHD)
+ SDQ_SDQ_Difficulties_Total: A composite measure summarizing overall difficulties across several behavioral domains
+ SDQ_SDQ_Emotional_Problems: Focuses on internal emotional difficulties such as anxiety or depression (social related)
+ SDQ_SDQ_Externalizing: Captures outward-directed behaviors such as hyperactivity, impulsivity, and conduct issues
+ SDQ_SDQ_Generating_Impact: This might reflect the overall impact of the child’s behavioral problems on their social and academic life
+ SDQ_SDQ_Hyperactivity: Directly measures the hyperactive and impulsive behaviors central to many ADHD diagnoses (HIGHLY CORRELATED FEATURE)
+ SDQ_SDQ_Internalizing: Reflects inward-focused behaviors such as social withdrawal and anxiety
+ SDQ_SDQ_Peer_Problems: Assesses difficulties in interacting with peers
+ SDQ_SDQ_Prosocial: Evaluates positive social behaviors like empathy and cooperation

## 2.2 Categorical Features Visualization: Total of 12 features (already label encoded)

+ Basic_Demos_Enroll_Year: the year when the participant enrolled in the study (int) (NOT VERY IMPORTANT)
+ Basic_Demos_Study_Site: Location/site where the subject was assessed (NOT VERY IMPORTANT)
+ PreInt_Demos_Fam_Child_Ethnicity: Ethnic background of the child
+ PreInt_Demos_Fam_Child_Race: Race of the child
+ MRI_Track_Scan_Location: Where the MRI was performed
+ Barratt_Barratt_P1_Edu: education of the parent 1 (ORDINAL)
+ Barratt_Barratt_P1_Occ: occupation of parent 1 (ORDINAL)
+ Barratt_Barratt_P2_Edu: education of the parent 2 (ORDINAL)
+ Barratt_Barratt_P2_Occ: occupation of parent 2 (ORDINAL)
+ Laterality_Category: Categorical brain lateralization: left, middle, or right
+ ColorVision_Level: Categorical encoding of color vision test (BINARY)
+ APQ_CP_is_high: Is Corporal Punishment score high (>6) (BINARY)

## 2.3 fMRI Connectome Matrices
+ Dimensionality Reduction: Apply techniques like Principal Component Analysis (PCA), Independent Component Analysis (ICA), or Uniform Manifold Approximation and Projection (UMAP) to the flattened connectome matrices before applying other feature selection methods or feeding them into models like Logistic Regression or standard Tree-based algorithms. Deep Learning models might handle the high dimensionality better directly.

# 3. Feature Engineering (Model-Agnostic)

## Inter-feature correlation 
You might keep only one feature from a highly correlated group, perhaps the one more strongly correlated with the targets or based on domain knowledge. This is particularly important for Logistic Regression which is sensitive to multicollinearity.

+ Covariance matrix: symmetric and positive semidefinite and tells us about the spread of the data => Plot the correlation matrix

## Correlation with targets
+ (Numerical features): The Pearson's correlation coefficient is often used, which is a normalized version of covariance that ranges from -1 to +1, providing a more standardized measure of the strength and direction of the linear relationship. This indicator only captures the linear relationship between variables. It is worth noting that when categorical features are encoded and they are NOMIAL, a low p-value might not reflect fully its influence
Solution: non-linear transformation, interaction features, feature combinations (arithmetic), domain knowledge 

+ (Numerical and categorical features) Mutual information: Captures the statistical dependence between two random variables. Unlike correlation, MI can capture both linear and non-linear relationships. A higher MI score indicates a stronger dependency between the nominal feature and the categorical target.

## Statistical Tests
+ ANOVA F-test: For numerical features vs. each categorical target. Tests if the mean of the numerical feature differs significantly across the target groups (e.g., mean age for ADHD vs. non-ADHD). A significant p-value suggests relevance.

+ Chi-Squared Test: For categorical features (like `PreInt_Demos_Fam_Child_Race`, `Laterality_Category`) vs. each categorical target. Tests for independence between the feature and the target. A significant p-value suggests dependence/relevance.

## Solution
Variance Threshold: Remove features with very low variance. These features are nearly constant and thus provide little predictive information. Be cautious with numerical features if they haven't been scaled.

A feature might be considered important if it shows relevance (high correlation/MI, significant test) to either `ADHD_Outcome` or `Sex_F`. You can rank features based on their scores for each target and combine the rankings or set thresholds.

# 3+: Feature Engineering (Model-Specific)

+ Recursive Feature Elimination (RFE): Starts with all features, trains a model (e.g., Logistic Regression, SVM, or a Tree-based model), removes the least important feature(s) based on coefficients or feature importance scores, and repeats until the desired number of features is reached. The importance is evaluated based on the model's performance on a validation set using the weighted F1 score.

+ Sequential Feature Selection (SFS): Forward Selection: Starts with no features, iteratively adds the feature that results in the best model performance (using the weighted F1 score) until no further improvement is seen.
OR
Backward Elimination: Starts with all features, iteratively removes the feature whose removal least degrades (or most improves) model performance (using the weighted F1 score).

## Embedded Methods (Model-Integrated) 
+ L1 Regularization (Lasso): Used with linear models like Logistic Regression. Adds a penalty proportional to the absolute value of the coefficients. This forces some coefficients to become exactly zero, effectively1 removing those features from the model. You can train a Logistic Regression model with L1 penalty and select the features with non-zero coefficients.   

+ Tree-Based Feature Importance: Models like Random Forest, Gradient Boosting Machines (XGBoost, LightGBM, CatBoost) naturally compute feature importance scores during training (e.g., based on Gini impurity reduction or the number of times a feature is used to split). Train a multi-output tree-based model and use these importance scores to rank and select features. Features with low importance can be dropped.

# 4. Modeling 

+ Logistic Regression: likely to converge to a point due to its simplistic architecture, despite how many feature engineering you did 

+ MLP: The depth of the network allows for hierarchical feature learning, which is particularly useful for complex non-linearities. Each layer the model learns new features, but it is a black box and we as humans cannot interpret what those features are

+ Tree-based algorithm: Gradient Boosting Machines (GBM) (e.g., XGBoost, LightGBM, CatBoost): These are also ensemble methods that build trees sequentially, with each new tree trying to correct the errors made by the previous ones. They are highly effective at capturing intricate non-linear patterns and often achieve state-of-the-art performance

+ Support Vector Machines with non-linear kernels: By using kernel functions (like Radial Basis Function (RBF), polynomial, or sigmoid), SVMs can implicitly map the data into a higher-dimensional space where it might become linearly separable. This allows them to learn complex non-linear decision boundaries in the original feature space

# 5. Inference / Evaluation